In [1]:
import numpy as np
import pandas as pd
import math

In [5]:
data_raw = pd.read_stata('nshap_w1_w2_w3_core_disposition.dta')

## Data cleaning

In [23]:
# Since the data contains original cohort and follow-up cohort, we need to filter out the follow-up cohort
print("Total observations: {}".format(len(data_raw)))
data_w1 = data_raw[~data_raw['w1WEIGHT_ADJ'].isnull()]
print("Removed follow-up {} respondents without W1 observations".format(len(data_raw) - len(data_w1)))
data = data_w1[~(data_w1['w3DECEASED'].isnull() & data_w1['w2DECEASED'].isnull())]
print("Removed follow-up {} respondents without disposition statuses".format(len(data_w1) - len(data)))
data['deceased'] = data['w3DECEASED'].fillna(data['w2DECEASED'])
print("Sample frame has {} respondents".format(len(data)))

Total observations: 6069
Removed follow-up 3064 respondents without W1 observations
Removed follow-up 0 respondents without disposition statuses
Sample frame has 3005 respondents


In [24]:
data.head()

,ID,SAMPLETYPE,w3DISP_GRP,w3DECEASED,ID_as_string,w2DISP_GRP,w2DECEASED,w1FI_ID,w1PATH,w1VERSION,...,w3IWDESCRIBE3,w3IWDESCRIBE4,w3IWDESCRIBE5,w3AREACOMFORT,w3AREASAFE,w3AREAAMENITIES,w3AREAPRESENCE,w3CASEDIF,_merge,deceased
0,100005,Returning respondent,Other (known/presumed alive),Alive,100005,Interviewed,Alive,866808,1.0,version 1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matched (3),Alive
1,100033,Returning respondent,Interviewed,Alive,100033,Interviewed,Alive,169164,3.0,version 3,...,1 (no traffic),2,1 (no smell or air pollution),1 (I felt comfortable),1 (I felt safe),2,5 (They did not notice my presence),not at all difficult,Matched (3),Alive
3,100080,Returning respondent,Interviewed,Alive,100080,Interviewed,Alive,281481,6.0,version 2,...,1 (no traffic),5 (houses far apart),1 (no smell or air pollution),1 (I felt comfortable),1 (I felt safe),2,3,not at all difficult,Matched (3),Alive
5,100154,Returning respondent,Interviewed,Alive,100154,Interviewed,Alive,024499,4.0,version 3,...,1 (no traffic),2,1 (no smell or air pollution),1 (I felt comfortable),1 (I felt safe),3,5 (They did not notice my presence),not at all difficult,Matched (3),Alive
7,100203,Returning respondent,Poor health,Alive,100203,Interviewed,Alive,289103,3.0,version 3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matched (3),Alive
